

Inversion of the Moho depth using gravity gradient data
----------------------------------------

This Jupyter Notebook allows the inversion of gravity gradient or vertical gravity data to obtain the Moho depth. 
The script is designed to define a laterally variable density, based on tectonic units. The tectonic units can be changed manually.

First the gravity data and tectonic units are loaded, as well as preconditions for the inversion are set.
Second, the inversion is performed once using a single value of Reference Moho depth and density contrast, in order to calculate a reference Jacobian Matrix.
Finally, the variable density contrast is integrated and the inverted Moho depth and density contrast are calculated. 

For official use, please cite the paper: Haas, P., Ebbing J., Szwillus W. - Sensitivity analysis of gravity gradient inversion of the Moho depth – A case example for the Amazonian Craton, Geophysical Journal International, 2020.

You need Python 3 to run this notebook. This notebok has been tested on a Windows machine.





#### Load the Python packages that are required for the inversion

In [1]:
%matplotlib inline

import numpy as np
from grad_inv_functions import *
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
from matplotlib import path
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import time
import progressbar

#### Set parameters for the inversion

In [2]:
lon_min=-80 #Define coordinates of study area. If coordinates are changed, farfield effect has to be deactivated
lon_max=-30
lat_min=-30
lat_max=20

dx=1.0 # define grid cell size (should be 1.0 degree)
dy=1.0

data_name="Moho_Amazonia_1deg_real_data_sed"
# Define Prefix name of output files 

mode="single" # Choose between"single" and "FTG". "Single" forward calculates the gravitational effect 
              #of the inverted component only, 
              # "Full" calculates the FTG and gz of the predicted Moho of the inverted component.
component="gzz" # choose between all gradient components and gz (in XYZ coordinate system)
farfield="yes" # choose between "yes" and "no" to include gravitational effect of farfield isostasy
sediments="no" # choose between "yes" and "no" to include gravitational effect of sediments

dens_min,dens_max,dens_inc=250.0,500.0,50
j=np.arange(-30000, -30001, -2500) # select range and increment of reference Moho depth (should be constant)
k=np.arange(dens_min,dens_max+0.1,dens_inc) # select range and increment of density contrast
l=np.arange(1,1.1,1) #  select number of iterations
beta=np.arange(1e-4,1.1e-4,1e-4) # select range and increment of smoothing parameter (should be constant)


# Define some initial conditions
dens_init=k[0]
lon=np.arange(lon_min,lon_max+dx,dx)
lat=np.arange(lat_min,lat_max+dy,dy)
longr,latgr = np.meshgrid(lon,lat)

#  Prepare vectors and grids for tesseroid model
lon=longr.flatten() 
lat=latgr.flatten()
height_km=225 
area = (lat.min(), lat.max(), lon.min(), lon.max())
shape=np.array((abs(lat_max-lat_min)/dy+1,abs(lon_max-lon_min)/dx+1))
shape=shape.astype(int)

#### Load and prepare the data

Seismic stations and tectonic units can be changed manually and are cut to the study area. The function "Doperator" calculates the smoothing matrix for the inversion using 2nd-order Tikhonov regularization.
The tectonic units are used to create density combinations and have to be organized as clusters.  


In [3]:
# Load and prepare seismic data
seismic_stations=np.loadtxt("Seismic_Moho_USGS_global.txt")
seismic_stations=cut_data_to_study_area(seismic_stations,area)
seismic_stations=seismic_stations[np.lexsort((seismic_stations[:,0],seismic_stations[:,1]))]

#Prepare Regularization
dmatrix_init=Doperator(shape,dx,dy)
smooth_matrix=np.eye(lon.size)*beta
dmatrix=dmatrix_init*beta

# Load and prepare clustering analysis
# The tectonic units have to be in xyz-format
tec_units=np.loadtxt('SL2013sv_Cluster_1d_interp.xyz')
tec_units=cut_data_to_study_area(tec_units,area)
tec_units=tec_units[np.lexsort((tec_units[:,0],tec_units[:,1]))]
number_of_units=len(np.unique(tec_units[:,2]))
dens_mat=create_density_combinations(k,number_of_units)
#dens_mat=np.ones((3,6))*400  # This line can be activated to manually define the number of density combinations
rms_matrix=np.zeros((k.size**number_of_units,number_of_units+5))

#### Inversion with single reference Moho depth and constant density contrast

The inversion of the gravity data is now performed once to obtain the reference Jacobian Matrix. Optionally, the inverted Moho depth as well as the residual field can be stored. 

In [4]:
save_fields="no" #select "yes" to save the computed fields as gridded data-files

for refmoho in j: # Loop over reference Moho depth

    # Load and prepare initial Moho depth
    moho=np.loadtxt("IsoMoho_Airy_initial_1degree.txt")
    moho=cut_data_to_study_area(moho,area)
    moho=moho[np.lexsort((moho[:,0],moho[:,1]))]
    moho_input=moho[:,2]
    moho_start=np.copy(moho)
    
    # Create flat starting model
    moho=moho[:,2]*0-30000

    for iteration in l: # Loop over iteration number

        print("iteration=",iteration)
        
        # Prefix for file name
        prefix = "%s_%s_farfield_%s_%dkm_it%d" % (data_name,component,farfield,
                                                             height_km,iteration)
        
        # Prepare layers of tesseroid model and define density contrast
        (moho,moho_top,moho_bottom,moho_top_shift,moho_bottom_shift,
         reference)=construct_layers_for_gradient_inversion(refmoho,moho,area,dx,dy) 
        density=np.ones(lon.shape[0])*dens_init
        density[moho < reference] = -dens_init

        # Load the gravitational data
        # IMPORTANT: To invert other components than gzz, the gravity data in the source code have to be 
        # changed manually!
        data=load_grav_data(farfield,sediments,area)
        bouguer=data[:,2]

        # Create the Jacobian Matrix
        J,J_shift,bigger_mat=create_Jacobian(longr,latgr,component,
                                moho_top,moho_bottom,moho_top_shift,moho_bottom_shift,height_km,density)
        
        # Invert the Moho depth and calculate residual fields 
        # NOTE: To save the calculated fields and the Moho depth "save_fields" has to be activated!
        moho_final,bouguer_fit=invert_and_calculate(prefix,moho,bouguer,J,J_shift,bigger_mat,dmatrix,
                                                    save_fields,mode,shape)

iteration= 1.0


[                                                                        ] N/A%

Calculate 1st Jacobian


[=                                                                       ]   2%

KeyboardInterrupt: 

#### Set preconditions for lateral variable density contrast

In [ ]:
# Reload the gravity data
data=load_grav_data(farfield,sediments,area)
bouguer=np.copy(data[:,2])

# Define boundaries for calculation of RMS values
inc=1.9
bound=np.array(([lon_min+inc, lat_min+inc],[lon_min+inc, lat_max-inc],
                [lon_max-inc, lat_max-inc],[lon_max-inc, lat_min+inc]))
bound=path.Path(bound)
inside=bound.contains_points(data[:,0:2])*1
data[:,2]=data[:,2]*inside
save_fields="no"

# Define multiplicator of density contrasts for Jacobian Matrix
mult=np.copy(density)
mult=mult/dens_init

# Create progress bar
bar = progressbar.ProgressBar(maxval=len(dens_mat[:,0]), \
widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
print("Invert density contrasts")
print("Number of iterations:",len(dens_mat[:,0]))

#### Inversion with a laterally variable density contrast

In [ ]:
bar.start()
for i in range(0,len(dens_mat[:,0])): # Loop over all possible combinations of density contrasts
#for i in range(13380,13420):# select only a single or a range of density contrasts
    #print(i, end='\r')
    bar.update(i+1)

    for ii in range(1,number_of_units+1): # Transfer density combination to density contrasts of grid
        ind=np.where(tec_units[:,2]==ii)
        density[ind]=dens_mat[i,ii-1]
    
    # Update the density contrasts for Jacobian Matrix
    density=np.abs(density)
    density=density*mult

    # Weight the Jacobian Matrix
    J_new,J_shift_new=weight_Jacobian(J,J_shift,density,dens_init)   
    
    # Invert the Moho depth, NOTE: calculation of fields is deprecated in this case
    moho_final,bouguer_fit=invert_and_calculate(prefix,moho,bouguer,J_new,J_shift_new,bigger_mat,dmatrix,
                                                save_fields,mode,shape)

    moho_int=moho_final*1000

    # interpolate inverted Moho depth onto seismic stations
    moho_resid_points,interp_arr=interp_regular_grid_on_irregular_database(area,dx,moho_int,seismic_stations)

    # Compute residual Moho depth (grid)
    moho_resid_grid=moho_input-moho_final*-1 # residual Moho depth
    
    
    # Create RMS values of residual field and residual Moho depths and store them in a matrix
    rms_matrix=create_rms_matrix(rms_matrix,data,moho_resid_points,moho_resid_grid,i,bouguer_fit)
    rms_matrix[i,5:5+number_of_units]=dens_mat[i,0:number_of_units]
    
    # Check fit to seismic stations and save the corresponding inverted Moho depth and density contrasts
    if i>1:      
        if rms_matrix[i,1]<np.amin(rms_matrix[0:i,1]):
            moho_save=np.array((lon,lat,moho_final*-1)).T
            density_save=np.array((lon,lat,density)).T
            np.savetxt(prefix+"_inverted_Moho_best_fit.txt",moho_save,delimiter=' ',fmt='%1.2f')
            np.savetxt(prefix+"_inverted_densities_best_fit.txt",density_save,delimiter=' ',fmt='%1.1f')
    
    #print(rms_matrix[i,:])

bar.finish()

#### Prepare and save the RMS Matrix

In [ ]:
rms_matrix[:,3]=beta
rms_matrix[:,4]=refmoho
print(rms_matrix.shape)
print(rms_matrix[np.argmin(rms_matrix[:,1]),:])
np.savetxt("rms_matrix.txt",rms_matrix,delimiter=' ',fmt='%1.4f',
           header="Gravity fit, Moho RMS Points, Moho RMS Grid, Beta, Ref_Moho, Density contrasts of tectonic units")

#### Plot distribution of the Moho models

The plot extensions can be changed by the user

In [ ]:
no_models=1000 # select number of models to plot 

In [ ]:
I = np.argsort(rms_matrix[:,1]) 
rms_matrix=rms_matrix[I,:]
bin_size=0.2
lower=np.around(np.amin(rms_matrix[:,1]))-bin_size
upper=np.around(np.amax(rms_matrix[:,1]))+bin_size
bins=np.arange(lower,upper,bin_size)
fig, ax = plt.subplots()
plt.hist(rms_matrix[:,1], bins = bins,color='lightgrey')
plt.hist(rms_matrix[0:no_models,1], bins = bins) 
plt.xticks(np.arange(lower,upper,bin_size*2),fontsize=12,fontname="Arial")
plt.yticks(np.arange(0,1001,100),fontsize=12,fontname="Arial")
ax.xaxis.set_minor_locator(MultipleLocator(bin_size))
ax.yaxis.set_minor_locator(MultipleLocator(50))
plt.xlabel('Error of Moho model [$km$]',fontsize=12,fontname="Arial")
plt.ylabel('Counts',fontsize=12,fontname="Arial")
#plt.savefig("test_rmss2.png", bbox_inches = 'tight', pad_inches = 0,dpi=300,transparent=True)
plt.show()

#### Prepare inverted density contrasts

In [ ]:
values=np.around(np.arange(0,number_of_units+0.1-1,1))
dens_counts=np.zeros((0,2))
for i in values: 
    unique, counts = np.unique(rms_matrix[0:no_models,int(i)+5], return_counts=True)
    val=np.array((unique,counts)).T
    if val[0,0]!=dens_min:
        fill=np.arange(dens_min,val[0,0],dens_inc)
        fill=np.array((fill,np.zeros(len(fill)))).T
        val=np.concatenate((fill,val),axis=0)
    if val[-1,0]!=dens_max:
        fill=np.arange(val[-1,0]+dens_inc,dens_max+0.1,dens_inc)
        fill=np.array((fill,np.zeros(len(fill)))).T
        val=np.concatenate((val,fill),axis=0)
    dens_counts=np.append(dens_counts,val,axis=0)

#### Plot distribution of inverted density contrasts

In [ ]:
rgb_values=np.array(([61.0,204.0,75.0],[0.0,64.0,255.0],[167.0,186.0,209.0],
                     [204.0,3.0,0.0],[255.0,187.0,51.0],[178.0,152.0,152.0]))/255

fig,ax = plt.subplots()
labels=["Craton","Precambrian Belt","Phanerozoic","Ridges","Oceanic","Oldest Oceanic"]
for i in range(0,len(values)):
    comb=dens_counts[i*len(k):len(k)*(i+1),:]
    plt.plot(comb[:,0], comb[:,1], '--', color=(rgb_values[i]),label=labels[i])
    plt.plot(comb[:,0], comb[:,1], '.',markersize=12, color="white", markeredgecolor=(rgb_values[i]))
    plt.plot(comb[np.argmax(comb[:,1]),0],comb[np.argmax(comb[:,1]),1],'.',markersize=15,color=(rgb_values[i]))

ax.yaxis.set_minor_locator(MultipleLocator(100))
plt.xticks(k,fontsize=10,fontname="Arial")
plt.yticks(np.arange(0,1001,200),fontsize=10,fontname="Arial")
plt.xlabel('Density contrast [kgm$^{-3}$]',fontsize=10,fontname="Arial")
plt.ylabel('Counts',fontsize=10,fontname="Arial")
font = font_manager.FontProperties(family='Arial',size=9)
plt.legend(prop=font)
#plt.savefig("test_rmss3.png", bbox_inches = 'tight', pad_inches = 0,dpi=300)
plt.show()